In [7]:
import paho.mqtt.client as mqtt

def on_connect(client, userdata, flags, rc):
    print("Connected with result code "+str(rc))

client = mqtt.Client()
client.on_connect = on_connect

client.connect("mqttcar.cloud.shiftr.io", 443, 60)

0

In [1]:
import sys
!{sys.executable} -m pip install --upgrade https://github.com/aparrish/bezmerizing/archive/master.zip

  Using cached https://github.com/aparrish/bezmerizing/archive/master.zip
  Created wheel for bezmerizing: filename=bezmerizing-0.1.1-py3-none-any.whl size=17284 sha256=049c2ef373a19030dc0a4544d6cc2ba98bff650fe0f237f394f4846284b2deda
  Stored in directory: /private/var/folders/3v/lxt4vwb95zs2ttfnbk5g_pch0000gn/T/pip-ephem-wheel-cache-lymv7u00/wheels/74/5a/7d/4a62c2490b6c6061916e41ae070caaed12664893aef1a56d7c
Successfully built bezmerizing
  Attempting uninstall: bezmerizing
    Found existing installation: bezmerizing 0.1.1
    Uninstalling bezmerizing-0.1.1:
      Successfully uninstalled bezmerizing-0.1.1


In [2]:
from flat import font
from flat.command import moveto, quadto, curveto, lineto, closepath
from bezmerizing import Polyline, Path
from copy import copy

In [3]:
f1 = font.open('VT323-Regular.ttf')
f2 = font.open('Orbitron-Black.ttf')


In [4]:
def glyphcommands(f, ch):
    return Path([copy(cmd) for cmd in f.glyph(f.charmap[ord(ch)])])
def advancefor(f, ch):
    return f.advances[f.charmap[ord(ch)]]


In [64]:
# input a string and a font file => writes glyph commands in that font to Arduino file
def getCommandsFromFont(text, font): 
    t = open('/Users/yonatanrozin/Documents/Arduino/MOL_Final/printerRobot/letterCommands.h', "w")
    
    startX = 0
    
    numLets = len(text)

    letterCommands = []

    for i in range(numLets): #for every letter:
        letter = glyphcommands(font, text[i]).commands #get glyph commands for that letter
        for j in range(len(letter)): #for each command: 
            anyRepeat = False
#             print("move" in str(letter[j]))
            if (hasattr(letter[j], "x") and hasattr(letter[j], "y")): #if command is NOT closepath:
                if ("move" in str(letter[j])): #if command is moveto:
                    letterCommands.append([None,None]);
                x = letter[j].x+startX
                y = letter[j].y
                letterCommands.append([x,y])
            if (letter[j] == letter[-1]): #if command is the last in the letter:
                startX+=advancefor(font, text[i]) #offset next letter's position according to advance
    
    lastX = None
    lastY = None
#     for c in range(len(letterCommands)):
#         print(letterCommands[c][0] == letterCommands[c+1][0] and letterCommands[c+1][0] == letterCommands[c+2][0], letterCommands[c][0])
        
        
    #quasi-infinity extremes for min/max values
    xMin = 10000
    yMin = 10000
    yMax = -10000
    
    #get min x,y coords
    for cmd in letterCommands:
        if (cmd[0] != None): #for commands that are not PEN UP:
            if (cmd[0] < xMin): #if command's x-coord is less than min
                xMin = cmd[0] #x-coord = new min
            if (cmd[1] < yMin): #repeat for y-coords
                yMin = cmd[1]
    
    #offset all coords by min x,y so NEW min x,y = 0,0 (for JavaScript)
    for cmd in letterCommands:
        if (cmd[0] != None): #for commands that are not PEN UP:
            cmd[0] -= xMin 
            cmd[1] -= yMin
            if cmd[1] > yMax: #if normalized y-coord is greater than max:
                yMax = cmd[1] #y-coord = new max
    
    yRatio = (1000/yMax) #get ratio of 1000:yMax
    
    for cmd in letterCommands: #multiply ALL coordinates by ratio
        if (cmd[0] != None):
            cmd[0] = int(cmd[0] * yRatio)
            cmd[1] = int(cmd[1] * yRatio)
    
    
    
    letterCommands = str(letterCommands).replace("None","null")
    commandsString = "int letterCommands[][2] = " + letterCommands.replace('[', "{").replace("]","}").replace("null","NULL")+";"
    print(letterCommands)
    t.write(commandsString)
    t.close();
    

        
getCommandsFromFont("1234567890", f2)


[[null, null], [539, 1000], [327, 1000], [327, 324], [281, 381], [0, 381], [319, 1], [539, 1], [539, 1000], [null, null], [1600, 1000], [619, 1000], [619, 613], [679, 470], [822, 411], [1342, 411], [1388, 411], [1388, 366], [1388, 259], [1388, 213], [1342, 213], [877, 213], [832, 213], [832, 259], [832, 294], [619, 294], [619, 202], [679, 60], [822, 1], [1398, 1], [1540, 60], [1600, 202], [1600, 423], [1540, 565], [1398, 625], [877, 625], [832, 625], [832, 671], [832, 742], [832, 787], [877, 787], [1600, 787], [1600, 1000], [null, null], [2545, 1000], [1968, 1000], [1825, 940], [1765, 798], [1765, 721], [1977, 721], [1977, 742], [1977, 787], [2023, 787], [2488, 787], [2533, 787], [2533, 742], [2533, 649], [2533, 603], [2488, 603], [1929, 603], [1929, 391], [2450, 391], [2496, 391], [2496, 345], [2496, 259], [2496, 213], [2450, 213], [2023, 213], [1977, 213], [1977, 259], [1977, 291], [1765, 291], [1765, 202], [1825, 60], [1968, 1], [2507, 1], [2649, 60], [2708, 202], [2708, 402], [2706